### How to scrape the USGS Publications Warehouse

###### By Martin Palkovic

##### Have you ever needed to search a website many times, and the thought of doing it manually crushed your soul?
##### If so, check out the workflow below to automate that process!

In [ ]:
#import libraries
import re, requests, time
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs

In [ ]:
#list generated from ArcGIS feature class. This list could contain anything you want to search for on USGS pubs
quads = ['Red Creek Ranch', 'Four J Rim', 'Scrivner Butte', 'Chicken Creek SW', 'Chicken Creek SE', 'Cow Creek Reservoir SW', 'Upper Powder Spring', 'Powder Mountain', 'Rotten Springs', 'McPherson Springs', 'Poison Basin', 'Willow Creek Butte', 'Baggs', 'Beaver Basin', 'Dixon', 'Sparks', 'Savery', 'Sugarloaf Butte', 'Grieve Reservoir', 'Hiawatha', 'Cottonwood Rim', 'Coffeepot Spring', 'Fletcher Peak', 'Powder Wash', 'Solomon Creek', 'Reservoir Draw', 'Dudley Creek', 'Blackhall Mountain', 'The Nipple NE', 'Trent Creek', 'Bighole Butte', 'Elkhorn Point', 'Thornburgh Gulch', 'Swallow Canyon', 'Horatio Rock', 'Pole Gulch', 'Lodore School', 'Foxpark', 'Fortification NE', 'Woods Landing', 'Big Joe Basin', 'Bakers Peak', 'Jelm Mountain', 'Irish Canyon', 'Fly Creek', 'Downey Lakes', 'G Spring', 'Johnson Ranch', 'Tumble Mountain', 'Sheepherder Springs', 'Best Ranch', 'Shield Mountain', 'Dale Creek', 'Lang Spring', 'Elkhorn Mountain', 'Buford', 'The Nipple', 'West Fork Lake', 'Granite', 'Davis Peak', 'The Nipple SE', 'Emkay', 'Pearl', 'Borie', 'Mayberry Spring', 'Cheyenne South', 'Independence Mountain', 'Great Divide', 'Hoy Mountain', 'Altvan', 'Northgate', 'East Timberlake Creek', 'Campstool', 'Canyon of Lodore North', 'Kings Canyon', 'Arcola', 'Fortification', 'Old Roach', 'Jack Springs', 'Carpenter', 'Freeman Reservoir', 'Crazy Mountain', 'Pine Bluffs SW', 'Vermillion Mesa', 'Pine Bluffs SE', 'Buck Point', 'Sand Creek Pass', 'Sheephead Basin', 'Bushnell SW', 'Eaton Reservoir', 'Bears Ears Peaks', 'Bushnell SE', 'Clay Buttes', 'Diamond Peak', 'Meaden Peak', 'Kimball SW', 'Cherokee Park', 'Kimball SE', 'Sevenmile Draw', 'Hahns Peak', 'Potter SW', 'Virginia Dale', 'Ninemile Hill', 'Farwell Mountain', 'Potter SE', 'Table Mountain', 'Easterly Airport', 'Mount Zirkel', 'Bald Mountain', 'Round Butte', 'Lorenzo', 'Boettcher Lake', 'Carr West', 'Sidney Municipal Airport', 'Adobe Springs', 'Sidney SE', 'Carr East', 'Lake John', 'Iron Springs', 'Lodgepole SW', 'Jones Hole', 'Chalk Bluffs West', 'Cowdrey', 'Lodgepole SE', 'Craig NW', 'Chalk Bluffs East', 'Chappell', 'Canyon of Lodore South', 'Eagle Hill', 'Chappell SE', 'Hereford NW', 'Craig NE', 'Barton', 'Zenobia Peak', 'Shipman Mountain', 'Hereford', 'Big Springs', 'McInturf Mesa', 'Glendevey', 'Grover North', 'Greystone', 'Grover NE', 'Slide Mountain', 'Deadman', 'Limestone Hill', 'Dolan Spring', 'South Bald Mountain', 'Quaker Mountain', 'Vim School', 'Lone Mountain', 'Red Feather Lakes', 'Pilot Knob', 'Battle Canyon', 'Haystack Gulch', 'Dipper Spring', 'Peck Mesa', 'Clark', 'Chimney Canyons', 'Livermore Mountain', 'Sunbeam', 'Floyd Peak', 'Kirchnavy Butte', 'Livermore', 'Padroni NW', 'Mount Ethel', 'Maybell', 'Buckeye', 'Peetz', 'Pitchpine Mountain', 'Carr SW', 'Haystack Butte', 'Lay', 'Twin Buttes', 'Dover', 'Delaney Butte', 'Lay SE', 'Stuntz Reservoir', 'Julesburg Reservoir', 'Eastman Creek South', 'Walden', 'Sedgwick', 'Pine Ridge', 'Eastman Creek SE', 'Hells Canyon', 'Ovid', 'Gould NW', 'Julesburg', 'Reno Reservoir', 'Craig', 'Tanks Peak', 'Venango NW', 'Johnny Moore Mountain', 'Hereford SE', 'Venango NE', 'Ralph White Lake', 'Rawah Lakes', 'Grover South', 'Haystack Rock', 'Rock Spring Gulch', 'Grover SE', 'Boston Peak', 'Indian Water Canyon', 'Pawnee Buttes', 'Kinikinik', 'Hooker Mountain', 'Gatehook Spring', 'Twelvemile Mesa', 'Rustic', 'Wolf Mountain', 'Avalo', 'Big Narrows', 'Cross Mountain Canyon', 'Avalo SE', 'Mad Creek', 'Westplains', 'Poudre Park', 'Cedar Knob', 'Rocky Peak', 'North Sterling Reservoir', 'Laporte', 'Padroni', 'Buffalo Pass', 'Juniper Mountain', 'Wellington', 'Iliff', 'Teal Lake', 'Cobb Lake', 'Juniper Hot Springs', 'Proctor', 'Crook', 'Nunn', 'Coalmont', 'Horse Gulch', 'Snake John Reef', 'Tamarack Ranch', 'Antelope Reservoir', 'MacFarlane Reservoir', 'Marks Butte', 'Round Bottom', 'Purcell', 'Plug Hat Rock', 'Julesburg SW', 'Owl Ridge', 'Julesburg SE', 'Baker Draw', 'Castor Gulch', 'Lazy Y Point', 'Gould', 'Venango SW', 'Briggsdale', 'Venango', 'Breeze Mountain', 'Clark Peak', 'Keota NW', 'Skull Creek', 'Hayden', 'Keota', 'Chambers Lake', 'M F Mountain', 'Raymer NW', 'Comanche Peak', 'Mount Harris', 'Raymer NE', 'Elk Springs', 'Pingree Park', 'Milner', 'Stoneham NW', 'Crystal Mountain', 'Wapiti Peak', 'Stoneham NE', 'Cow Creek', 'Wild Horse Lake', 'Buckhorn Mountain', 'Citadel Plateau', 'Steamboat Springs', 'Atwood NE', 'Horsetooth Reservoir', 'Sterling North', 'Mount Werner', 'Price Creek', 'Fort Collins', 'Galien', 'Rabbit Ears Peak', 'Timnath', 'Easton Gulch', 'Uhler Ranch', 'Fleming', 'Severance', 'Spicer Peak', 'Axial', 'Dinosaur', 'Haxtun West', 'Eaton', 'Buffalo Peak', 'Haxtun East', 'Monument Butte', 'Mellen Hill', 'Galeton', 'Holyoke NW', 'Rand', 'Holyoke NE', 'Cornish', 'Hamilton', 'Rangely NE', 'Jack Creek Ranch', 'Amherst', 'Fosston', 'Amherst NE', 'Pagoda', 'Mount Richthofen', 'Dutch Girl Lake', 'Cactus Reservoir', 'Hayden Gulch', 'Keota SE', 'Fall River Pass', 'Divide Creek', 'Buckingham', 'Trail Ridge', 'Dunckley', 'Raymer', 'Rough Gulch', 'Estes Park', 'Rattlesnake Butte', 'Stoneham', 'Glen Haven', 'Smizer Gulch', 'Stoneham SE', 'Oak Creek', 'Drake', 'Willard', 'Indian Valley', 'Blacktail Mountain', 'Atwood', 'Masonville', 'Sterling South', 'White Rock', 'Walton Peak', 'Loveland', 'Reiradon Hill', 'Lake Agnes', 'Devils Hole Gulch', 'Windsor', 'Leroy', 'Saint Petersburg', 'Bracewell', 'Whiteley Peak', 'Ninemile Gap', 'Walsh Knolls', 'Rockland', 'Greeley', 'Hyannis Peak', 'Haxtun SE', 'Thornburgh', 'Banty Point', 'Kersey', 'Paoli', 'Parkview Mountain', 'Holyoke', 'Barnesville', 'Sleepy Cat Peak', 'Rangely', 'Radial Mountain', 'Amherst SW', 'Point of Rocks', 'Slide Creek', 'Amherst SE', 'Bowen Mountain', 'Gillam Draw', 'Greasewood Lake', 'Pagoda Peak', 'Sunken Lake', 'Grand Lake', 'Calamity Ridge', 'Judson Hills', 'Dunckley Pass', 'McHenrys Peak', 'Peace Valley School', 'Barcus Creek', 'Longs Peak', 'Sand Point', 'Dead Horse Springs', 'Barcus Creek SE', 'Panorama Peak', 'Antelope Springs', 'Yampa', 'Pinewood Lake', 'Messex', 'White River City', 'Green Ridge', 'Merino', 'Carter Lake Reservoir', 'Buckskin Point', 'Buffalo Springs Ranch NW', 'Gore Mountain', 'Berthoud', 'Buffalo Springs Ranch NE', 'Tyler Mountain', 'Meeker', 'Johnstown', 'Glacken Hill', 'New Haven', 'Hinman Reservoir', 'Milliken', 'Rattlesnake Mesa', 'Weaver Ridge', 'Clarkville', 'La Salle', 'Gunsight Pass', 'Clarkville NE', 'Sawmill Mountain', 'Banta Ridge', 'Valley View School', 'Fiddler Peak', 'Corral Peaks', 'Fawn Creek', 'Hardin', 'Wauneta NE', 'Water Canyon', 'Cabin Creek', 'Alvin NW', 'Dearfield', 'Lost Park', 'Alvin NE', 'Trail Mountain', 'Philadelphia Creek', 'Masters', 'Ripple Creek', 'Shadow Mountain', 'Orchard', 'Sagebrush Hill', 'Weldona', 'Devils Causeway', 'Isolation Peak', 'Wolf Ridge', 'Fort Morgan', 'Allenspark', 'Orno Peak', 'Brush West', 'Square S Ranch', 'Raymond', 'Brush East', 'Trapper', 'Lyons', 'Merino SW', 'Greasewood Gulch', 'Toponas', 'Merino SE', 'Hygiene', 'Segar Mountain', 'Lynx Pass', 'Buffalo Springs Ranch', 'Longmont', 'Buffalo Springs Ranch SE', 'Gore Pass', 'Lo 7 Hill', 'Gowanda', 'Burdett', 'Kremmling', 'Lone Star', 'Platteville', 'Dragon', 'Veatch Gulch', 'Clarkville SW', 'Milton Reservoir', 'Junction Butte', 'Clarkville SE', 'Big Beaver Reservoir', 'Texas Creek', 'Klug Ranch', 'Old Baldy', 'Parshall', 'Buford', 'Tampa', 'Wauneta', 'Texas Mountain', 'Hot Sulphur Springs', 'Alvin SW', 'Roggen', 'Oyster Lake', 'Alvin', 'Granby', 'White Coyote Draw', 'Omar', 'Big Marvine Peak', 'Strawberry Lake', 'Wiggins', 'Black Cabin Gulch', 'Vallery', 'Trappers Lake', 'Monarch Lake', 'Yankee Gulch', 'Lamb', 'Ward', 'Dome Peak', 'Round Top', 'Rock School', 'Gold Hill', 'Burns North', 'Miller Ranch', 'Boulder', 'Pinneo', 'Jessup Gulch', 'Blue Hill', 'Fremont Butte', 'Niwot', 'No Name Ridge', 'McCoy', 'Akron', 'Erie', 'Platner', 'Radium', 'Thirteenmile Creek', 'Frederick', 'Otis', 'Sheephorn Mountain', 'Fort Lupton', 'Hyde', 'Davis Canyon', 'Red Elephant Point', 'Yuma North', 'Hudson', 'King Creek', 'Triangle Park', 'Yuma NE', 'East Evacuation Creek', 'Keenesburg', 'Eckley NW', 'Battle Mountain', 'Meadow Creek Lake', 'Prospect Valley', 'Eckley NE', 'Big Foundation Creek', 'Sylvan Reservoir', 'Wray NW', 'South Roggen', 'Blair Mountain', 'Wray NE', 'Brushy Point', 'Bottle Pass', 'Wiggins SW', 'Deep Lake', 'Fraser', 'Hoyt', 'Razorback Ridge', 'Adena', 'Sweetwater Lake', 'East Portal', 'Figure Four Spring', 'Vallery SE', 'Nederland', 'Sugarloaf Mountain', 'Huey Ranch', 'Bull Fork', 'Tungsten', 'Burns South', 'Gary', 'Eldorado Springs', 'Rago', 'Cutoff Gulch', 'Castle Peak', 'Pinneo SE', 'Louisville', 'McCarthy Gulch', 'State Bridge', 'Akron SW', 'Lafayette', 'Akron SE', 'Lava Creek', 'Rio Blanco', 'Eastlake', 'Snyder Lake', 'Piney Peak', 'Brighton', 'Otis SE', 'Rat Hole Ridge', 'Horse Mountain', 'Yuma South', 'Mile High Lakes', 'Mount Powell', 'Rifle Falls', 'Schramm', 'Baxter Pass', 'Horse Creek', 'Squaw Creek', 'Eckley', 'Deep Creek Point', 'Sunnydale', 'Robb', 'Douglas Pass', 'Ute Peak', 'Wray', 'Living Springs', 'Adams Lake', 'Laird', 'Calf Canyon', 'Byers Peak', 'Leader NW', 'Carbonate', 'Berthoud Pass', 'Leader', 'Henderson Ridge', 'Potty Brown Creek', 'Broken Rib Creek', 'Empire', 'Desert Gulch', 'Shamrock', 'Central City', 'Dotsero', 'Woodrow NW', 'Mount Blaine', 'Black Hawk', 'Gypsum', 'Woodrow', 'Ralston Buttes', 'Circle Dot Gulch', 'Antelope Creek West', 'Eagle', 'Antelope Creek East', 'Golden', 'Forked Gulch', 'Wolcott', 'Elba', 'Arvada', 'Elba NE', 'Edwards', 'Anvil Points', 'Commerce City', 'De Nova NW', 'Vail West', 'Montbello', 'Jim Canyon', 'De Nova NE', 'Rifle', 'Beverly Grove', 'Denver International Airport', 'Vail East', 'Silt', 'Carbonera', 'Heartstrong', 'Manila', 'Willow Lakes', 'Vernon NW', 'New Castle', 'Bennett', 'Howard Canyon', 'Vernon', 'Dillon', 'Beecher Island NW', 'Roper School', 'Storm King Mountain', 'Garvey Canyon', 'Willow Creek Ranch', 'Loveland Pass', 'Leader SW', 'Glenwood Springs', 'Grays Peak', 'Leader SE', 'Middle Dry Fork', 'Shoshone', 'Poison Springs', 'Georgetown', 'The Saddle', 'Shamrock SE', 'Idaho Springs', 'Cottonwood Pass', 'Wetzel Creek', 'Long Point', 'Squaw Pass', 'Suicide Mountain', 'Woodlin School', 'Evergreen', 'Red Pinnacle', 'Dry Gulch', 'The Seven Hermits', 'Antelope Creek SE', 'Morrison', 'Parachute', 'Fulford', 'Elba SW', 'Fort Logan', 'Elba SE', 'Grouse Mountain', 'Rulison', 'Englewood', 'De Nova', 'Minturn', 'Bar X Wash', 'Fitzsimons', 'North Mamm Peak', 'De Nova SE', 'Abarr', 'Coal Creek', 'Red Cliff', 'Hunter Mesa', 'Badger Wash', 'Abarr SE', 'Watkins', 'Vail Pass', 'Vernon SW', 'Gibson Gulch', 'Highline Lake', 'Strasburg NW', 'Wildcat Canyon', 'Frisco', 'Beecher Island', 'Strasburg', 'Center Mountain', 'Ruby Lee Reservoir', 'Armel', 'Keystone', 'Byers', 'Cattle Creek', 'Montezuma', 'Peoria', 'Corcoran Peak', 'Carbondale', 'Noonen Reservoir', 'Mount Evans', 'Winter Flats', 'Cottonwood Valley North', 'Harris Park', 'Leon', 'Last Chance NW', 'Wagon Track Ridge', 'Meridian Hill', 'Toner Reservoir', 'Last Chance', 'Conifer', 'De Beque', 'Lindon', 'Red Creek', 'Lindon NE', 'Indian Hills', 'Housetop Mountain', 'Crooked Creek Pass', 'Anton', 'Littleton', 'Arickaree', 'Hawxhurst Creek', 'Mount Jackson', 'Highlands Ranch', 'Cope NW', 'Mount of the Holy Cross', 'Bitter Creek Well', 'South Mamm Peak', 'Parker', 'Cope', 'Joes', 'Piney Creek', 'Pando', 'Hightower Mountain', 'Ruby Canyon', 'Adler Creek', 'Watkins SE', 'Copper Mountain', 'Spring Canyon', 'Flatiron Mountain', 'Mack', 'Strasburg SW', 'Idalia', 'Breckenridge', 'Bonny Reservoir North', 'Strasburg SE', 'Quaker Mesa', 'Fruita', 'Boreas Pass', 'Hale Ponds', 'Byers SW', 'Stony Ridge', 'Jefferson', 'Corcoran Point', 'Deer Trail', 'Mount Sopris', 'Noonen Reservoir SW', 'Mount Logan', 'Round Mountain', 'Cottonwood Valley South', 'Basalt', 'Shawnee', 'Last Chance SW', 'Cameo', 'Bailey', 'Woody Creek', 'Lusto Springs', 'Mesa', 'Pine', 'Lindon SW', 'Ruedi Reservoir', 'Shaw', 'Platte Canyon', 'Molina', 'Meredith', 'Thurman', 'Kassler', 'Anton SE', 'Collbran', 'Nast', 'Sedalia', 'Cope SW', 'Westwater', 'Homestake Reservoir', 'Vega Reservoir', 'Castle Rock North', 'Cope SE', 'Joes SW', 'Ponderosa Park', 'Leadville North', 'Sieber Canyon', 'Porter Mountain', 'Kirk', 'Cabin Gulch', 'Climax', 'Idalia SW', 'Spruce Mountain', 'Battleship Rock', 'Kiowa NW', 'Idalia SE', 'Alma', 'Bonny Reservoir South', 'Kiowa NE', 'Elk Knob', 'Colorado National Monument', 'Como', 'Bonny Reservoir SE', 'Bijou', 'Placita', 'Milligan Lakes', 'Grand Junction', 'Cattle Gulch', 'Redstone', 'Agate', 'Observatory Rock', 'Clifton', 'Barking Dog Spring', 'Capitol Peak', 'Topaz Mountain', 'T Draw', 'Palisade', 'Windy Peak', 'Highland Peak', 'Walks Camp Park', 'Lands End', 'Green Mountain', 'Arriba NW', 'Aspen', 'Deckers', 'Arriba NE', 'Mesa Lakes', 'Thimble Rock', 'Flagler NW', 'Devils Head', 'Grand Mesa', 'Flagler NE', 'Mount Champion', 'Dawson Butte', 'Seibert NW', 'Marble Canyon', 'Mount Massive', 'Leon Peak', 'Castle Rock South', 'Seibert NE', 'Stratton NW', 'Leadville South', 'Russellville Gulch', 'Bieser Creek', 'Chalk Mountain', 'Tuttle', 'Elizabeth', 'Mount Sherman', 'Settlement', 'Electric Mountain', 'Payne Wash', 'Kiowa', 'Burlington NE', 'Fairplay West', 'Kanorado NW', 'Bull Mountain', 'Big Gulch', 'Glade Park', 'Fairplay East', 'Kanorado NE', 'Bijou SW', 'Chair Mountain', 'Elkhorn', 'Island Mesa', 'Kuhns Crossing', 'Marble', 'Beuck Draw', 'Eagle Rock', 'Whitewater', 'River Bend', 'Snowmass Mountain', 'Farnum Peak', 'Juniata Reservoir', 'Limon', 'McCurdy Mountain', 'Maroon Bells', 'Genoa West', 'Indian Point', 'Cheesman Lake', 'Genoa East', 'Hayden Peak', 'Westcreek', 'Arriba', 'Hells Kitchen', 'New York Peak', 'Flagler SW', 'Dakan Mountain', 'Cedaredge', 'Independence Pass', 'Flagler', 'Larkspur', 'Flagler Reservoir', 'Steamboat Mesa', 'Mount Elbert', 'Dry Creek', 'Greenland', 'Seibert', 'Vona', 'Granite', 'Cherry Valley School', 'Two V Basin', 'Gray Reservoir', 'Stratton', 'Elbert', 'South Peak', 'Bethune', 'Bowie', 'Fish Creek', 'Bijou Basin', 'Burlington', 'Jones Hill', 'Somerset', 'Peconic', 'Fondis', 'Snyder Flats', 'Garo', 'Kanorado', 'Ramah North', 'Paonia Reservoir', 'Jacks Canyon', 'Hartsel', 'Simla', 'Marcellina Mountain', 'Sulphur Mountain', 'Matheson', 'Triangle Mesa', 'Matheson NE', 'Oh-be-joyful', 'Glentivar', 'Dominguez', 'Lake', 'Tarryall', 'Gothic', 'Barron Creek', 'Point Creek', 'Hackett Mountain', 'Pearl Pass', 'Hugo', 'Signal Butte', 'Sevenmile Ranch', 'North Delta', 'Italian Creek', 'East of Sevenmile Ranch', 'Mount Deception', 'Orchard City', 'Pieplant', 'South of Flagler', 'Palmer Lake', 'South of Flagler Reservoir', 'Dolores Point North', 'Winfield', 'Lazear', 'Monument', 'South of Seibert', 'Mount Harvard', 'Alpine Ranch NW', 'Black Forest', 'Gateway', 'Hotchkiss', 'Alpine Ranch NE', 'Eastonville', 'Harvard Lakes', 'Paonia', 'South of Bethune', 'Pine Mountain', 'Peyton', 'South of Burlington', 'Marmot Peak', 'Minnesota Pass', 'Calhan', 'Mount Sunflower NW', 'Casto Reservoir', 'Antero Reservoir', 'Mount Sunflower NE', 'Ramah South', 'West Beckwith Mountain', 'Keith Creek', 'Antero Reservoir NE', 'Alta Vista', 'Anthracite Range', 'Guffey NW', 'Matheson SW', 'Escalante Forks', 'Matheson SE', 'Mount Axtell', 'Spinney Mountain', 'Good Point', 'Long Creek', 'Elevenmile Canyon', 'Crested Butte', 'Lake SE', 'Roubideau', 'Lake George', 'Cement Mountain', 'Hugo SW', 'Divide', 'Clifford', 'Delta', 'Matchless Mountain', 'Bledsoe Ranch', 'Woodland Park', 'Olathe NW', 'Taylor Park Reservoir', 'Bellyache Creek', 'Cascade', 'Big Spring Creek', 'Dolores Point South', 'Tincup', 'Black Ridge', 'Pikeview', 'North of Big Spring', 'Mount Yale', 'Falcon NW', 'Juanita Arch', 'Alpine Ranch SW', 'Grand View Mesa', 'Alpine Ranch', 'Falcon', 'Buena Vista West', 'Crawford', 'Calamity Mesa', 'Rock Creek', 'Haegler Ranch', 'East of Rock Creek', 'Buena Vista East', 'Mount Guero', 'Holcolm Hills', 'Uncompahgre Butte', 'Mount Sunflower SW', 'Castle Rock Gulch', 'Mount Sunflower', 'Rush NW', 'Big Soap Park', 'Snipe Mountain', 'Agate Mountain', 'Holtwold Store', 'West Elk Peak', 'Dicks Peak', 'Kutch NW', 'Kelso Point', 'Squirrel Creek', 'Kutch', 'Thirtynine Mile Mountain', 'Cottonwood Basin', 'Punkin Center NW', 'Witcher Mountain', 'Flat Top', 'Beckman Lake', 'Camel Back', 'Wrights Reservoir', 'Almont', 'Kinney Lake', 'Cripple Creek North', 'Hoovers Corner', 'Boyero', 'Crystal Creek', 'Schafer Reservoir', 'Pikes Peak', 'Olathe', 'Fairview Peak', 'Sanders Ranch', 'Manitou Springs', 'Buckeye Reservoir', 'Kit Carson NW', 'Red Rock Canyon', 'Cumberland Pass', 'Colorado Springs', 'Big Spring', 'Saint Elmo', 'Roc Creek', 'Elsmere', 'Eureka Creek North', 'Grizzly Ridge', 'Landsman Hill', 'Corral Bluffs', 'Mount Antero', 'Cathedral Peak', 'Red Canyon', 'Cheyenne Wells NW', 'Ellicott', 'Nathrop', 'Cheyenne Wells NE', 'X Lazy F Ranch', 'Atkinson Creek', 'Big Springs Ranch', 'Arapahoe NW', 'Cameron Mountain', 'Arapahoe NE', 'Yoder', 'Little Soap Park', 'Windy Point', 'Gribbles Park', 'Rush', 'West Elk Peak SW', 'Black Mountain', 'Kutch SW', 'Starvation Point', 'McIntosh Mountain', 'Kutch SE', 'Thirtyone Mile Mountain', 'Moore Mesa', 'Punkin Center', 'Cover Mountain', 'Gunnison', 'Stanley Gulch', 'Davis Point', 'High Park', 'Signal Peak', 'McKenzie Draw', 'Cripple Creek South', 'Dry Creek Basin', 'Rock Basin', 'Parlin', 'Aroya', 'Big Bull Mountain', 'Montrose West', 'Pitkin', 'Wild Horse', 'Mount Big Chief', 'Ray Mesa', 'Sorrento', 'Montrose East', 'Whitepine', 'Cheyenne Mountain', 'Kit Carson', 'Paradox', 'Garfield', 'Fountain', 'Cerro Summit', 'Eureka Creek South', 'Firstview', 'Fountain NE', 'Maysville', 'Cimarron', 'Davis Mesa', 'Cheyenne Wells SW', 'Hanover NW', 'Salida West', 'Cheyenne Wells', 'Curecanti Needle', 'Uravan', 'Hanover NE', 'Arapahoe', 'Salida East', 'Arapahoe SE', 'Truckton', 'Sapinero', 'Nucla', 'Jack Hall Mountain', 'Truckton NE', 'Carpenter Ridge', 'Waugh Mountain', 'Big Bucktail Creek', 'Sanborn Reservoir', 'Big Mesa', 'Peace Valley', 'Hall Gulch', 'Ute', 'Forder', 'Iris NW', 'Gribble Mountain', 'Karval', 'Antone Spring', 'Rice Mountain', 'Iris', 'Hubbard Lake', 'Pryor Creek', 'Cooper Mountain', 'Barrel Springs Draw', 'Houston Gulch', 'Stacy Lakes Draw', 'Phantom Canyon', 'Government Springs', 'Doyleville', 'Galatea NE', 'Mount Pittsburg', 'Lisbon Gap', 'Arsenic Lake', 'Colona', 'Sargents', 'Timber Mountain', 'Lewis Lake', 'Anderson Mesa', 'Pahlone Peak', 'Buckhorn Lakes', 'Buttes', 'East of Lewis Lake', 'South of Firstview', 'Fountain SE', 'Mount Ouray', 'Bull Canyon', 'Washboard Rock', 'South Ladder Creek', 'Hanover', 'Poncha Pass', 'South of Cheyenne Wells', 'Lost Lake', 'Naturita NW', 'Hanover SE', 'Lake Albert NW', 'Wellsville', 'Colony Township NW', 'Edison School', 'Poison Draw', 'Naturita', 'Howard', 'Truckton SE', 'Gateview', 'Arkansas Mountain', 'Redvale', 'Cockleburr Springs', 'Powderhorn', 'Walker Point', 'Echo', 'Norwood', 'Sharp Lake', 'Spring Hill Creek', 'McIntyre Hills', 'Metz Springs', 'Sanborn Park', 'Royal Gorge', 'Sawtooth Mountain', 'Scott Draw', 'Hotchkiss Reservoir', 'Canon City', 'Bluff Spring', 'Razor Creek Dome', 'Galatea SW', 'Florence', 'Horsefly Peak', 'West Baldy', 'Galatea', 'Pierce Gulch', 'Summit Point', 'Arsenic Lake SW', 'Ridgway', 'Sargents Mesa', 'Stone City', 'Dunlap Ranch', 'Horse Range Mesa', 'Chester', 'Dallas', 'Steele Hollow', 'Oswald Ranch', 'North of Chivington', 'Pinon', 'Bonanza', 'Hamm Canyon', 'Courthouse Mountain', 'North of Brandon', 'Bar J H Ranch', 'Whale Hill', 'North of Sheridan Lake', 'Sheep Mountain', 'Gypsum Gap', 'North Avondale NE', 'Lake Albert', 'Bushnell Peak', 'Colony Township West', 'Alpine Plateau', 'Highlands Church', 'Basin', 'Coaldale', 'Grandview School', 'Powderhorn Lakes', 'Barkelew Draw', 'Cotopaxi', 'Ninemile Spring', 'Rudolph Hill', 'Windmill Lake', 'Hillside', 'Oak Hill', 'Box Springs', 'Rock Creek Park', 'Iron Mountain', 'Gurley Canyon', 'The Pinnacles', 'Curley Peak', 'Cold Spring Park', 'Trimble Lake', 'Placerville', 'Rockvale', 'Arlington NE', 'Cochetopa Park', 'Haswell', 'Florence SE', 'Sams', 'North Pass', 'Haswell NE', 'Hobson', 'Piute Knoll', 'Mount Sneffels', 'Hawkins', 'Trickle Mountain', 'Swallows', 'Eads', 'Egnar', 'Lake Mountain NE', 'Ouray', 'Northwest Pueblo', 'Alkali Lake', 'Chivington', 'Klondike Mine', 'Northeast Pueblo', 'Joe Davis Hill', 'Wetterhorn Peak', 'Brandon', 'Devine', 'Graveyard Gulch', 'Sheridan Lake', 'Uncompahgre Peak', 'Dawson Draw', 'North Avondale', 'Stuart', 'Villa Grove', 'Towner', 'Lake City', 'Boone Hill', 'McKenna Peak', 'Valley View Hot Springs', 'Grandview School SE', 'Cannibal Plateau', 'North Mountain', 'Electric Peak', 'Antelope Mesa', 'Mineral Mountain', 'Nero Hill', 'Beckwith Mountain', 'Lone Cone', 'Lake Henry', 'Stewart Peak', 'Westcliffe', 'Beaver Park', 'Todd Point', 'Mount Tyndall', 'Elk Park', 'Houston Lakes', 'Little Cone', 'Hardscrabble Mountain', 'Arlington', 'Saguache Park', 'Wetmore', 'Long Lake', 'Gray Head', 'Grouse Creek', 'Haswell SE', 'Owl Canyon', 'Northdale', 'Telluride', 'Rose Ranch', 'Lake Mountain', 'Beulah NE', 'Swede Lake', 'Dove Creek', 'Laughlin Gulch', 'Ironton', 'Southwest Pueblo', 'Neenoshe Reservoir', 'Chivington SE', 'Saguache', 'Southeast Pueblo', 'Secret Canyon', 'Handies Peak', 'Sheridan Lake SW', 'Vineland', 'Hickey Bridge', 'Sheridan Lake SE', 'Redcloud Peak', 'The Glade', 'Avondale', 'Lake Devore', 'Moffat North', 'Lake San Cristobal', 'Towner SE', 'Nepesta', 'Glade Mountain', 'Mirage', 'Fowler', 'Slumgullion Pass', 'South Mountain', 'Rito Alto Peak', 'Olney Springs', 'Baldy Cinco', 'Horn Peak', 'Ordway', 'Groundhog Reservoir', 'Sugar City', 'San Luis Peak', 'Aldrich Gulch', 'Groundhog Mountain', 'Meredith Hill', 'Rosita', 'Halfmoon Pass', 'Lewis Ranch', 'Dolores Peak', 'Deer Peak', 'Mesa Mountain', 'McIntosh Ranch', 'Saint Charles Peak', 'Bishop Ranch', 'Mount Wilson', 'Bowers Peak', 'Tree Top Ranch', 'Beulah', 'Burnt Cabin Creek', 'Ophir', 'Lookout Mountain', 'Lubers', 'Muldoon Hill', 'McClave', 'Champagne Spring', 'Lime Creek', 'Silverton', 'Verde School', 'Wiley', 'Swede Corners', 'May Valley', 'Cahone', 'Goat Butte', 'Howardsville', 'Granada NW', 'Doyle Bridge', 'Harrence Lake', 'Granada NE', 'Pole Creek Mountain', 'Doe Canyon', 'Chicos Well', 'Holly NW', 'Moffat South', 'Finger Mesa', 'Holly NE', 'Flying A Ranch', 'Narraguinnep Mountain', 'Sheds Camp', 'Hardesty Reservoir', 'Hermit Lakes', 'Willow Spring', 'Crestone', 'Elder', 'Bristol Head', 'Crestone Peak', 'Manzanola', 'Nipple Mountain', 'Rocky Ford', 'Creede', 'Beck Mountain', 'Clyde Lake', 'Cheraw', 'Devils Gulch', 'Wagon Wheel Gap', 'Hadley', 'Rico', 'Bear Creek', 'Pool Table Mountain', 'Cornelia', 'San Isabel', 'Hermosa Peak', 'Las Animas', 'Pine Cone Knob', 'Kreybill', 'Papoose Canyon', 'Rye', 'Engineer Mountain', 'Twin Mountains', 'Hasty', 'Colorado City', 'Prowers', 'Ruin Canyon', 'Snowdon Peak', 'Twin Mountains SE', 'Graneros Flats', 'Lamar West', 'La Garita', 'Pleasant View', 'Lamar East', 'Cedarwood', 'Storm King Peak', 'Carlton', 'Hog Ranch Canyon', 'Center North', 'Rio Grande Pyramid', 'Yellow Jacket', 'Granada', 'Red Top Ranch', 'Holly West', 'Deadman Camp SW', 'Weminuche Pass', 'Trimble Point', 'Holly East', 'Yellowbank Creek', 'Deadman Camp', 'Apishapa Bridge', 'Little Squaw Creek', 'Boggy Draw', 'Sand Camp', 'Timpas NW', 'Workman Creek', 'Liberty', 'Timpas NE', 'Stoner', 'Spar City', 'Hawley', 'Medano Pass', 'Wallace Ranch', 'La Junta', 'Creager Reservoir', 'Lake Humphreys', 'Thompson Arroyo', 'Orphan Butte', 'Gardner', 'South Fork West', 'Hackamore Ranch', 'Badito Cone', 'Elk Creek', 'Gilpin', 'South Fork East', 'Toonerville NE', 'Ruin Point', 'Hayden Butte', 'Electra Lake', 'Indian Head', 'High Rock', 'Huerfano Butte', 'Negro Canyon', 'Denny Lake', 'Mountain View Crest', 'Del Norte', 'Lascar', 'Cat Creek NW', 'Woods Canyon', 'Sevenmile Plaza', 'Cat Creek NE', 'Capps Springs', 'Columbine Pass', 'North Plum Creek NW', 'North Rattlesnake Butte', 'Center South', 'Arriola', 'Emerald Lake', 'North Plum Creek NE', 'Myers Canyon', 'Durkee Creek NW', 'Hooper West', 'Granite Lake', 'Dolores West', 'Sanford Hills', 'Durkee Creek NE', 'Hooper East', 'Snowden Lake', 'Cimarrona Peak', 'Dolores East', 'Medano Ranch', 'Timpas SW', 'Palomino Mountain', 'Zapata Ranch', 'Millwood', 'Timpas', 'South River Peak', 'La Junta SW', 'Mosca Pass', 'Rampart Hills', 'La Junta SE', 'Mount Hope', 'Red Wing', 'Higbee', 'La Plata', 'Little Sheep Mountain', 'Beaver Creek Reservoir', 'Turkey Canyon', 'Monument Hill', 'Farisita', 'Toonerville', 'Del Norte Peak', 'Wickiup Canyon', 'Toonerville SE', 'Black Hills', 'Hermosa', 'Horseshoe Mountain', 'Hand Springs', 'Walsenburg North', 'Bowdish Canyon', 'Dripping Spring', 'Lemon Reservoir', 'Dog Mountain', 'Maria Reservoir', 'Cat Creek', 'Battle Rock', 'Monte Vista', 'Vallecito Reservoir', 'Cucharas Reservoir', 'Gobblers Knob', 'Barrel Spring', 'South Rattlesnake Butte', 'Homelake', 'Mud Creek', 'Granite Peak', 'North Plum Creek SE', 'Jones Lake Spring', 'Mount Pleasant School', 'Two Butte Springs', 'Bear Mountain', 'Cortez', 'Sun Valley Ranch', 'Durkee Creek SE', 'Hooper SE', 'Delhi', 'Oakbrush Ridge', 'Point Lookout', 'Dry Lakes', 'Bloom', 'Pagosa Peak', 'Twin Peaks', 'Mancos', 'Sheep Canyon', 'Saddle Mountain', 'Packers Gap', 'Blanca Peak', 'Thompson Park', 'Riley Canyon', 'Wolf Creek Pass', 'Russell', 'Corbin Canyon', 'Hesperus', 'La Veta Pass', 'Elwood Pass', 'Rock Canyon', 'Durango West', 'La Veta', 'Ninaview', 'Summitville', 'Peters Nipple', 'Clay Ranch', 'Ritter Arroyo', 'Durango East', 'Jasper', 'Pipe Spring', 'Walsenburg South', 'Mariano Wash West', 'Floating W Ranch', 'Rules Hill', 'Greenie Mountain', 'Pryor', 'Hasser Ranch', 'Mariano Wash East', 'Fulcher Gulch', 'Ludwig Mountain', 'Pryor SE', 'Two Buttes Reservoir', 'Two Buttes NW', 'Little Dome', 'Waverly', 'Towaoc', 'Baldy Mountain', 'Plains Community', 'Hidden Valley Ranch', 'Alamosa West', 'Webb', 'Devil Mountain', 'Wetherill Mesa', 'Bates Lake', 'Lycan NE', 'Alamosa East', 'Thatcher', 'Chris Mountain', 'Moccasin Mesa', 'Baldy', 'Lockwood Arroyo', 'Pagosa Springs', 'Trail Canyon', 'Blanca', 'Stage Canyon', 'Jackson Mountain', 'O V Mesa', 'Fort Garland', 'Mormon Reservoir', 'Beaty Canyon', 'Blackhead Peak', 'Trinchera Ranch', 'Lost Canyon', 'Kline', 'McCarty Park', 'Summit Peak', 'Brown Canyon', 'Basin Mountain', 'Cuchara', 'Plug Hat Ranch', 'Platoro', 'Yellow Rock Point East', 'Walker Canyon', 'Spanish Peaks', 'Loma Linda', 'Red Mountain', 'Deora', 'Santa Clara', 'Sentinel Peak SW', 'McEndree Ranch', 'Gem Village', 'Terrace Reservoir', 'Aguilar', 'Big Rock Grange', 'Sentinel Peak SE', 'Centro', 'Bayfield', 'The Hogback', 'Horse Creek Springs', 'Two Buttes', 'Vega Corral', 'Capulin', 'Tanner Mesa', 'Pargin Mountain', 'Two Buttes SE', 'Seven Lakes Reservoir', 'La Jara', 'Lycan', 'Chimney Rock', 'Moqui Canyon', 'Tyrone', 'Lycan SE', 'Pikes Stockade', 'Lonetree Canyon', 'Brown Sheep Camp', 'Greasewood Canyon', 'Lasauses', 'Rock Crossing', 'Oakbrush Hill', 'Red Horse Gulch', 'Blanca SE', 'Doss Canyon North', 'Serviceberry Mountain', 'Johnson Canyon', 'Fort Garland SW', 'Redmesa', 'Plum Canyon', 'Harris Lake', 'Ojito Peak', 'Pinkerton Mesa', 'Icehouse Canyon', 'Trinchera Peak', 'Elephant Head Rock', 'Robbers Roost Canyon', 'Long Mountain', 'Cucharas Pass', 'Buck Canyon', 'Victoria Lake', 'Teec Nos Pos', 'Table Mesa', 'Herlick Canyon', 'Bondad Hill', 'Spectacle Lake', 'Pritchett NW', 'Gulnare', 'Sallies Spring', 'Ignacio', 'Harbord', 'La Jara Canyon', 'Delagua', 'Springfield West', 'Canal Creek', 'Vicente Canyon', 'Tiffany', 'Ludlow', 'Springfield East', 'Vilas North', 'Goshawk Dam', 'Hoehne', 'Skinney Rock', 'Allison', 'Walsh', 'Earl', 'Manassa', 'Bartlett', 'Carracas', 'Palmer Mesa', 'Model', 'Saunders', 'Manassa NE', 'Pagosa Junction', 'Lambing Spring', 'Heifer Point', 'Mesito Reservoir', 'Painted Canyon', 'Trujillo', 'Purgatory Canyon', 'San Acacio', 'Doss Canyon South', 'Edith', 'Humbar Spring', 'San Luis', 'La Plata', 'Villegreen', 'Chromo', 'Taylor Ranch', 'Adobe Downs Ranch', 'Cherry Canyon', 'El Valle Creek', 'Chama Peak', 'Kim North', 'Cedar Hill', 'Stonewall', 'Andrix', 'Archuleta Creek', 'Utleyville', 'Vigil', 'Mount Nebo', 'Cumbres', 'Lone Rock', 'Weston', 'Anastacio Spring', 'Pritchett', 'Osier', 'Madrid', 'Springfield SW', 'Fox Creek', 'Burnt Mesa', 'Trinidad West', 'Bisonte', 'Vilas South', 'Antonito', 'Trinidad East', 'Bancos Mesa NW', 'Walsh SE', 'Mooney Hills', 'Lobatos', 'Stonington', 'Bancos Mesa', 'Patterson Crossing', 'Stonington SE', 'Kiowa Hill', 'Carracas Canyon', 'Trinchera Cave', 'Sky Valley Ranch', 'Trementina Canyon', 'Wirt Canyon', 'Garcia', 'Box Ranch', 'Dulce', 'Miners Peak', 'Sanchez Reservoir', 'Tobe', 'Monero', 'La Valley', 'Dalerose Mesa', 'Culebra Peak', 'Chromo Mountain', 'Kim South', 'Torres', 'Chama', 'Pintada Creek', 'Tercio', 'Carrizo Mountain', 'West Fork Rio Brazos', 'Reader Lake', 'Little Pine Canyon', 'Toltec Mesa', 'Edler', 'Valdez', 'Campo NW', 'Bighorn Peak', 'Starkville', 'Campo NE', 'Moore Draw NW', 'Los Pinos', 'Fishers Peak', 'Moore Draw NE', 'Barela', 'Pinabetoso Peaks', 'Midway', 'Abeyta', 'Midway NE', 'La Segita Peaks NE', 'Trinchera', 'Ute Mountain', 'Branson', 'Costilla', 'Branson SE', 'Amalia', 'Pine Canyon', 'Cobert Mesa North', 'Big Costilla Peak', 'Jesus Canyon', 'The Wall', 'Dennis Canyon', 'Vermejo Park', 'Furnish Canyon West', 'Casa Grande', 'Furnish Canyon East', 'Big Hole Canyon', 'Caliente Canyon North', 'Tubs Springs', 'McWilliams Canyon', 'Campo SW', 'Tin Pan Canyon', 'Campo', 'Moore Draw SW', 'Raton', 'Moore Draw SE', 'Yankee', 'Midway SW', 'Dale Mountain', 'Midway SE', 'Trinchera Pass', 'Alps Mesa', 'Emery Peak', 'Cross L Ranch', 'Cobert Mesa South', 'Travesser Park', 'Tub Spring', 'Wedding Cake Butte', 'Goodson School', 'Kenton', 'Kenton NE', 'Autograph Cliff NW', 'Razor Blade Mesa', 'Keyes NW', 'Keyes NE', 'Sturgis NW', 'Sturgis']
quads = [s + ' Geologic Map Colorado' for s in quads]
print(quads[:5])

#Code I used to generate the list above in ArcGIS, which I copy and pasted in to this Jupyter Notebook. Note that a 'list'
#is a Python data structure, and it's important that the information you want to search for is in a Python list 

# import arcpy
# quad_list = []
# with arcpy.da.SearchCursor('Quads24k', 'QUADNAME') as cursor:
#     for row in cursor:
#         quad_list.extend(row)
# print(quad_list)

In [ ]:
#Search the USGS publications warehouse one time for every 24k quad in Colorado. If a map exists, append it to a new
#Python list, which I'll then copy in to a text file.
#Note that this workflow could be modified to download data for any type of publication, website, data, etc


#Chrome and the chrome driver need to be the same version
#Chrome browser v. 87
d = webdriver.Chrome(r'C:\Users\mpalkovic\Downloads\chromedriver_87.exe') # Chrome driver v. 87
#test = ['Clark Peak Geologic Map Colorado', 'Sulphur Mountain Geologic Map Colorado'] #I usually make sure this works before letting it go on thousands of things
count = 0 #if the program fails due to a connection error, comment this line and rerun
usgs_quads = [] #if the program fails due to a connection error, comment this line and rerun
for quad in quads[count:]:
    count = count + 1
    d.get('https://pubs.er.usgs.gov/')
    e = d.find_element_by_id('q-text')
    e.send_keys(quad)
    e.send_keys(Keys.ENTER)
    
    page = requests.get(d.current_url)
    soup = bs(page.text, 'html.parser')
    
    for i in soup.find_all('a', href = re.compile('publication')):
        try:
            if 'quadrangle'.lower() in i.string:
                if 'Colorado' in i.string:
                    maps = i.string.strip()
                    if maps not in usgs_quads:
                        usgs_quads.append(maps)
                        print(i.string)
        except:
            print(count)
            pass
   
    #time.sleep(5) #makes the loop wait 5 seconds between every iteration. you likely wont need this, I experimented with it to fix connection errors, it didn't work
print(usgs_24k[0:5]) 

In [ ]:
usgs_quads = list(set(usgs_quads)) #deletes duplicates. There shouldn't be any, but just in case
print(len(usgs_24k))

In [ ]:
#This removes some of the smaller scale maps that may have ended up on the list erroneously
new_q = []
for m in usgs_24k:
    if '1 degree by 2 degrees' not in m:
        new_q.append(m)
        
for m in new_q:
    if 'degree' in m:
        new_q.remove(m)

for m in new_q:
    if '30\' x 60\'' in m:
        new_q.remove(m)

print(len(new_q))
new_q[0:5]

In [117]:
usgs_quads = new_q
usgs_quads

['Geology of the Red Canyon quadrangle, Montrose county, Colorado',
 'Geologic map of the Pinewood Lake quadrangle, Boulder and Larimer Counties, Colorado',
 'Geologic map of the Canyon of Lodore North quadrangle, Moffat County, Colorado',
 'Geologic map of the Elk Springs quadrangle, Moffatt County, Colorado',
 'Map showing areas of selected potential geologic hazards in the Aspen Quadrangle, Pitkin County, Colorado',
 'Map showing inferred relative permeability of geologic materials in the Parker Quadrangle, Arapahoe and Douglas counties, Colorado',
 'Geologic and coal outcrop map of the Madrid Quadrangle, Las Animas County, Colorado',
 'Reconnaissance geologic map of parts of the Twin Peaks and Blanca Peak quadrangles, Alamosa, Costilla, and Huerfano counties, Colorado',
 'Geology of the Anderson Mesa quadrangle, Colorado',
 'Preliminary geologic map of the Rifle Quadrangle, Garfield County, Colorado',
 'Geologic map of the Niwot Quadrangle, Boulder County, Colorado',
 'Geologic map